<a href="https://colab.research.google.com/github/vhrique/ELF/blob/main/20250131_Analise_Sucupira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
planilhas = {
    '2021': 'https://drive.google.com/uc?export=download&id=1aY-35G2D8oFIwY9UtxaUP9K00Cz72Tf9',
    '2022': 'https://drive.google.com/uc?export=download&id=14QUdOCIMWLOL3De60GKPkvlgxzaMCDPz',
    '2023': 'https://drive.google.com/uc?export=download&id=1C2qkNL1N9B6Q6wUh33kbkQ8QRySnEfnZ',
    # '2024': 'https://drive.google.com/uc?export=download&id=1WLvc4wt8RsiN2Ob6UWjGq9SodFvQORCj',
}

In [87]:
df_dict = {}
for ano in planilhas:
    df_dict[ano] = pd.read_excel(planilhas[ano], sheet_name=None)

In [88]:
docentes_permanentes = []
for ano in df_dict:
    new_df = df_dict[ano]['Docentes'].query('Categoria == "PERMANENTE"')
    docentes_permanentes.extend(new_df['Nome Docente'].tolist())
docentes_permanentes = set(docentes_permanentes)

In [63]:
dfs = []
for ano in df_dict:
    new_df = df_dict[ano]['Projetos - Membros']
    new_df['Ano da Coleta'] = ano
    dfs.append(new_df)
df_projeto_membros = pd.concat(dfs).reset_index(drop=True)

In [64]:
dfs = []
for ano in df_dict:
    new_df = df_dict[ano]['Projetos - Financiadores']
    new_df['Ano da Coleta'] = ano
    dfs.append(new_df)
df_projeto_financiadores = pd.concat(dfs).reset_index(drop=True)

In [65]:
dfs = []
for ano in df_dict:
    new_df = df_dict[ano]['Linhas de Pesquisa - Projetos']
    new_df['Ano da Coleta'] = ano
    dfs.append(new_df)
df_projeto_linhas = pd.concat(dfs).reset_index(drop=True)

In [48]:
for i, row in df_projeto_financiadores.iterrows():
    df_projeto_financiadores.loc[i,'Nome do Projeto de Pesquisa'] = df_projeto_financiadores.loc[i,'Nome do Projeto de Pesquisa'].replace('"','')

for i, row in df_projeto_membros.iterrows():
    df_projeto_membros.loc[i,'Nome do Projeto de Pesquisa'] = df_projeto_membros.loc[i,'Nome do Projeto de Pesquisa'].replace('"','')

In [66]:
projetos_financiados = df_projeto_financiadores['Nome do Projeto de Pesquisa'].unique().tolist()
projetos_com_membros = df_projeto_membros['Nome do Projeto de Pesquisa'].unique().tolist()
projetos_com_lp = df_projeto_linhas['Nome do Projeto de Pesquisa'].unique().tolist()
projetos_todos = list(set(projetos_financiados + projetos_com_membros + projetos_com_lp))

In [68]:
df = pd.DataFrame({
    'Projeto': projetos_todos
})

In [69]:
for i, row in df.iterrows():
    df_query = df_projeto_membros.loc[df_projeto_membros["Nome do Projeto de Pesquisa"] == row.Projeto]
    df.loc[i, 'Discente'] = 'Discente' in df_query['Categoria do Membro do Projeto'].tolist()

In [70]:
for i, row in df.iterrows():
    df_query = df_projeto_financiadores.loc[df_projeto_financiadores["Nome do Projeto de Pesquisa"] == row.Projeto]
    df.loc[i, 'Financiado'] = 'OUTRO AUXÍLIO FINANCEIRO' in df_query['Natureza do Financiador'].tolist()

In [76]:
for i, row in df.iterrows():
    df_query = df_projeto_linhas.loc[df_projeto_linhas["Nome do Projeto de Pesquisa"] == row.Projeto]
    df.loc[i, 'CDIAC'] = 'CIÊNCIA DE DADOS, INTELIGÊNCIA ARTIFICIAL E CONTROLE' in df_query['Linha de Pesquisa'].tolist()
    df.loc[i, 'GP'] = 'GESTÃO DE OPERAÇÕES' in df_query['Linha de Pesquisa'].tolist()
    df.loc[i, 'IDP'] = 'INOVAÇÃO E DESENVOLVIMENTO DE PRODUTO' in df_query['Linha de Pesquisa'].tolist()
    df.loc[i, 'IAS'] = 'INTEGRAÇÃO E AUTOMAÇÃO DE SISTEMAS' in df_query['Linha de Pesquisa'].tolist()
df['LP'] = df[['CDIAC', 'GP','IDP','IAS']].any(axis=1)

In [82]:
G1 = df.query('Financiado == True').LP.mean()
G1

1.0

In [83]:
G2 = df.query('Financiado == True and LP == True').Discente.mean()
G2

0.96

In [102]:
G3 = 0
for docente in docentes_permanentes:
    for projeto in df.query('Financiado == True').Projeto.tolist():
        df_query = df_projeto_membros.loc[df_projeto_membros["Nome do Projeto de Pesquisa"] == projeto]
        if docente in df_query['Nome do Membro do Projeto'].tolist():
            G3 += 1
            break
G3 = G3 / len(docentes_permanentes)
G3

1.0

In [86]:
G4 = df[['CDIAC', 'GP','IDP','IAS']].any(axis=0).mean()
G4

1.0